In [ ]:
# Celle 1: Introduksjon og Import
"""
# RAG-Workshop Notebook

Denne notebooken demonstrerer en RAG-pipeline ved hjelp av Azure AI-teknologier. 
Du kan bruke enten en forhåndsindeksert AI Search-tjeneste eller lese PDF-filer fra data-mappen.
Du kan også redigere prompten i cellen under.
"""

# Importer nødvendige biblioteker

import os
import openai
import requests
import fitz  # PyMuPDF
from dotenv import load_dotenv

# For interaktivitet
from ipywidgets import widgets
from IPython.display import display

# Last inn miljøvariabler (hvis du har en .env-fil)
load_dotenv()

# API-konfigurasjon
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")

AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")

# Konfigurer OpenAI-klienten
openai.api_type = "azure"
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_version = "2023-07-01-preview"
openai.api_key = AZURE_OPENAI_KEY


In [ ]:
# Celle 2: Konfigurer prompt
"""
Rediger prompten nedenfor. Bruk {context} og {user_input} som plassholdere.
"""
# Du kan redigere denne variabelen direkte
prompt_template = (
    "Du er en ekspert på tekniske spesifikasjoner. Her er noen relevante utdrag fra dokumentasjonen:\n\n"
    "{context}\n\n"
    "Bruk denne informasjonen til å svare på følgende spørsmål:\n{user_input}"
)
print("Nåværende prompt-mal:")
print(prompt_template)


In [ ]:
# Celle 3: Funksjoner for henting av data og generering av svar

def search_ai_index(query, top_k=3):
    """Henter relevante dokumenter fra Azure AI Search."""
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX}/docs/search?api-version=2023-07-01"
    headers = {"Content-Type": "application/json", "api-key": AZURE_SEARCH_KEY}
    payload = {"search": query, "top": top_k}
    
    response = requests.post(url, headers=headers, json=payload)
    results = response.json()
    documents = [doc["content"] for doc in results.get("value", [])]
    return documents

def extract_text_from_pdfs(data_folder="data"):
    """Leser tekst fra alle PDF-er i data-mappen og returnerer dem som en liste."""
    all_text = []
    if not os.path.exists(data_folder):
        print(f"Mappen '{data_folder}' finnes ikke. Opprett mappen og legg til PDF-filer.")
        return all_text
    
    for filename in os.listdir(data_folder):
        if filename.endswith(".pdf"):
            file_path = os.path.join(data_folder, filename)
            print(f"Leser: {file_path}")
            doc = fitz.open(file_path)
            text = "\n".join([page.get_text() for page in doc])
            all_text.append(text)
    return all_text

def query_gpt_rag(user_input, use_pdf=False):
    """Kombinerer henting av dokumenter og sender prompt til GPT-4."""
    if use_pdf:
        retrieved_docs = extract_text_from_pdfs()
    else:
        retrieved_docs = search_ai_index(user_input)
    
    context = "\n\n".join(retrieved_docs[:3])  # Begrens til de første 3 dokumentene
    
    # Bruk den eksterne prompten som er definert i prompt_template
    prompt = prompt_template.format(context=context, user_input=user_input)
    
    response = openai.ChatCompletion.create(
        engine=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "Du er en ekspert AI som bruker eksterne dokumenter for å gi svar."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )
    
    return response["choices"][0]["message"]["content"]


In [ ]:
# Celle 4: Interaktiv testing

# Tekstfelt for spørsmål
question_input = widgets.Text(
    value='Skriv inn ditt spørsmål her...',
    placeholder='Ditt spørsmål',
    description='Spørsmål:',
    disabled=False
)

# Radioknapper for å velge datakilde
data_source = widgets.RadioButtons(
    options=['AI Search', 'PDF-er'],
    description='Kilde:',
    disabled=False
)

# Kjør-knapp
run_button = widgets.Button(description="Kjør RAG Pipeline")

output = widgets.Output()

def on_run_button_clicked(b):
    with output:
        output.clear_output()
        user_question = question_input.value
        use_pdf = (data_source.value == 'PDF-er')
        print("Kjører pipeline...\n")
        answer = query_gpt_rag(user_question, use_pdf=use_pdf)
        print("🤖 RAG-forbedret svar:\n")
        print(answer)

run_button.on_click(on_run_button_clicked)

display(question_input, data_source, run_button, output)
